<a href="https://colab.research.google.com/github/yaswhar/learning-genesis-sim/blob/main/Worm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install genesis-world -q
!pip install mediapy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
ICD_CONFIG_CONTENT = """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write(ICD_CONFIG_CONTENT)

In [ ]:
import genesis as gs
import numpy as np
import torch
import mediapy as media
from tqdm.notebook import tqdm

In [ ]:
gs.init(seed=0, precision="32", logging_level="debug")

[Genesis] [22:55:43] [INFO] ╭───────────────────────────────────────────────╮
INFO:genesis:~<╭───────────────────────────────────────────────╮>~
[Genesis] [22:55:43] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
INFO:genesis:~<│┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈>~ ~~~~<Genesis>~~~~ ~<┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│>~
[Genesis] [22:55:43] [INFO] ╰───────────────────────────────────────────────╯
INFO:genesis:~<╰───────────────────────────────────────────────╯>~
[Genesis] [22:55:43] [INFO] Running on [Tesla T4] with backend gs.cuda. Device memory: 14.74 GB.
INFO:genesis:Running on ~~<[Tesla T4]>~~ with backend ~~<gs.cuda>~~. Device memory: ~~<14.74>~~ GB.
[Genesis] [22:55:44] [DEBUG] [Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.11.11
DEBUG:genesis:[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.11.11
[Genesis] [22:55:44] [DEBUG] [Taichi] Starting on arch=cuda
DEBUG:genesis:[Taichi] Starting on arch=cuda
[Genesis] [22:55:44] [INFO] 🚀 Genesis initialized. 🔖 v

In [ ]:
# Create Scene
scene = gs.Scene(
    sim_options=gs.options.SimOptions(
        substeps=10,
        gravity=(0, 0, -9.8),
    ),
    viewer_options=gs.options.ViewerOptions(
        camera_pos=(1.5, 0, 0.8),
        camera_lookat=(0.0, 0.0, 0.0),
        camera_fov=40,
    ),
    mpm_options=gs.options.MPMOptions(
        dt=5e-4,
        lower_bound=(-1.0, -1.0, -0.2),
        upper_bound=(1.0, 1.0, 1.0),
    ),
    vis_options=gs.options.VisOptions(
        show_world_frame=True,
        visualize_mpm_boundary=True,
    ),
    show_viewer = False
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (1.5, 0.0, 0.8),
    lookat = (0, 0, 0),
    fov    = 40,
    GUI    = False,
)

[Genesis] [22:55:47] [INFO] Scene <943c257> created.
INFO:genesis:Scene ~~~<<943c257>>~~~ created.


In [ ]:
# Add Entities
scene.add_entity(
    morph=gs.morphs.Plane(),
    material=gs.materials.Rigid(
        coup_friction=5.0,
    ),
)

worm = scene.add_entity(
    morph=gs.morphs.Mesh(
        file="meshes/worm/worm.obj",
        pos=(0.3, 0.3, 0.001),
        scale=0.1,
        euler=(90, 0, 0),
    ),
    material=gs.materials.MPM.Muscle(
        E=5e5,
        nu=0.45,
        rho=10000.0,
        model="neohooken",
        n_groups=4,
    ),
    surface=gs.surfaces.Default(
        diffuse_texture=gs.textures.ImageTexture(
            image_path="meshes/worm/bdy_Base_Color.png",
        ),
    ),
)

[Genesis] [22:55:53] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <b3de0e6>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<0>~, uid: ~~~<<b3de0e6>>~~~, morph: ~<<gs.morphs.Plane>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [22:55:53] [INFO] Preprocessing geom idx 0.
INFO:genesis:Preprocessing geom idx ~~<0>~~.
[Genesis] [22:55:53] [INFO] Adding <gs.MPMEntity>. idx: 1, uid: <82c7183>, morph: <gs.morphs.Mesh(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/meshes/worm/worm.obj')>, material: <gs.materials.MPM.Muscle>.
INFO:genesis:Adding ~<<gs.MPMEntity>>~. idx: ~<1>~, uid: ~~~<<82c7183>>~~~, morph: ~<<gs.morphs.Mesh(file='/usr/local/lib/python3.11/dist-packages/genesis/assets/meshes/worm/worm.obj')>>~, material: ~<<gs.materials.MPM.Muscle>>~.
[Genesis] [22:55:53] [INFO] Sampling particles with pbs-32 sampler and generating `.ptc` file:
INFO:genesis:Sampling particles with ~<pbs-32>~ sampler and generating `.ptc

In [ ]:
# Build
scene.build()

[Genesis] [22:56:14] [INFO] Building scene <943c257>...
INFO:genesis:Building scene ~~~<<943c257>>~~~...
[Genesis] [22:56:28] [INFO] Compiling simulation kernels...
INFO:genesis:Compiling simulation kernels...
[Genesis] [22:57:35] [INFO] Building visualizer...
INFO:genesis:Building visualizer...


In [ ]:
# Set Muscle
def set_muscle_by_pos(robot):
    if isinstance(robot.material, gs.materials.MPM.Muscle):
        pos = robot.get_state().pos
        n_units = robot.n_particles
    elif isinstance(robot.material, gs.materials.FEM.Muscle):
        pos = robot.get_state().pos[robot.get_el2v()].mean(1)
        n_units = robot.n_elements
    else:
        raise NotImplementedError

    pos = pos.cpu().numpy()
    pos_max, pos_min = pos.max(0), pos.min(0)
    pos_range = pos_max - pos_min

    lu_thresh, fh_thresh = 0.3, 0.6
    muscle_group = np.zeros((n_units,), dtype=int)
    mask_upper = pos[:, 2] > (pos_min[2] + pos_range[2] * lu_thresh)
    mask_fore = pos[:, 1] < (pos_min[1] + pos_range[1] * fh_thresh)
    muscle_group[mask_upper & mask_fore] = 0  # upper fore body
    muscle_group[mask_upper & ~mask_fore] = 1  # upper hind body
    muscle_group[~mask_upper & mask_fore] = 2  # lower fore body
    muscle_group[~mask_upper & ~mask_fore] = 3  # lower hind body

    muscle_direction = np.array([[0, 1, 0]] * n_units, dtype=float)

    robot.set_muscle(
        muscle_group=muscle_group,
        muscle_direction=muscle_direction,
    )


set_muscle_by_pos(worm)

In [ ]:
# Run
scene.reset()
frames = []
for i in range(1000):
    actu = np.array([0, 0, 0, 1.0 * (0.5 + np.sin(0.005 * np.pi * i))])

    worm.set_actuation(actu)
    scene.step()
    frames.append(cam.render()[0])

[Genesis] [23:08:49] [INFO] Resetting Scene <943c257>.
INFO:genesis:Resetting Scene ~~~<<943c257>>~~~.
[Genesis] [23:08:49] [INFO] Running at 0.03 FPS.
INFO:genesis:Running at ~<0.03>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.03 FPS.
INFO:genesis:Running at ~<0.03>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.03 FPS.
INFO:genesis:Running at ~<0.03>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.04 FPS.
INFO:genesis:Running at ~<0.04>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.04 FPS.
INFO:genesis:Running at ~<0.04>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.04 FPS.
INFO:genesis:Running at ~<0.04>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.04 FPS.
INFO:genesis:Running at ~<0.04>~ FPS.
[Genesis] [23:08:50] [INFO] Running at 0.04 FPS.
INFO:genesis:Running at ~<0.04>~ FPS.
[Genesis] [23:08:51] [INFO] Running at 0.05 FPS.
INFO:genesis:Running at ~<0.05>~ FPS.
[Genesis] [23:08:51] [INFO] Running at 0.05 FPS.
INFO:genesis:Running at ~<0.05>~ FPS.
[Genesis] [23:08:51] [INFO]

In [ ]:
media.show_video(frames, fps=100)